# Multi-Model Hyperparameter Tuning with MLflow and Optuna

XGBoost hyperparameter optimization using Optuna with MLflow child runs.
Trains separate models per product category for electronics sales prediction.

In [ ]:
import math
import numpy as np
import pandas as pd
import xgboost as xgb
import mlflow
import optuna
from sklearn.metrics import mean_squared_error

print(f"MLflow version: {mlflow.__version__}")
print(f"Optuna version: {optuna.__version__}")
print(f"XGBoost version: {xgb.__version__}")

## Configuration

In [ ]:
# Experiment settings
experiment_name = "electronics-sales-tuning"
n_trials = 50

# Data settings
n_rows_per_category = 1000
test_size = 0.25
seed = 42

# Categories to train (None = all)
categories_to_train = None  # or ["smartphones", "laptops"]

## Setup MLflow Experiment

In [ ]:
def get_or_create_experiment(experiment_name: str) -> str:
    """Get existing experiment or create new one."""
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment:
        return experiment.experiment_id
    return mlflow.create_experiment(experiment_name)

experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_id=experiment_id)
print(f"Experiment: {experiment_name} (ID: {experiment_id})")

## Load Electronics Sales Dataset

In [ ]:
from scripts.generate_electronics_sales_data import (
    get_category_datasets,
    prepare_features,
    PRODUCT_CATEGORIES,
)

# Load datasets per category
category_datasets = get_category_datasets(
    n_rows_per_category=n_rows_per_category,
    seed=seed,
)

# Filter categories if specified
if categories_to_train:
    category_datasets = {k: v for k, v in category_datasets.items() if k in categories_to_train}

print(f"Categories: {list(category_datasets.keys())}")
for cat, df in category_datasets.items():
    print(f"  {cat}: {len(df)} rows, target mean={df['units_sold'].mean():.0f}")

In [ ]:
# Prepare train/valid splits for each category
category_splits = {}
for cat, df in category_datasets.items():
    train_x, valid_x, train_y, valid_y = prepare_features(df, test_size=test_size, seed=seed)
    category_splits[cat] = {
        "train_x": train_x,
        "valid_x": valid_x,
        "train_y": train_y,
        "valid_y": valid_y,
        "dtrain": xgb.DMatrix(train_x, label=train_y),
        "dvalid": xgb.DMatrix(valid_x, label=valid_y),
    }
    print(f"{cat}: Train={train_x.shape}, Valid={valid_x.shape}")

## Visualization Functions

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_correlation_with_target(df, target_col="units_sold", save_path=None):
    """Plot correlation of features with target."""
    numeric_df = df.select_dtypes(include=[np.number])
    correlations = numeric_df.corr()[target_col].drop(target_col).sort_values()
    
    colors = sns.diverging_palette(10, 130, as_cmap=True)
    color_mapped = correlations.map(colors)
    
    sns.set_style("whitegrid", {"axes.facecolor": "#c2c4c2"})
    fig = plt.figure(figsize=(10, 6))
    plt.barh(correlations.index, correlations.values, color=color_mapped)
    plt.title(f"Correlation with {target_col}", fontsize=14)
    plt.xlabel("Correlation Coefficient")
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150)
    plt.close(fig)
    return fig

def plot_feature_importance(model, booster="gbtree"):
    """Plot XGBoost feature importance."""
    importance_type = "weight" if booster == "gblinear" else "gain"
    
    fig, ax = plt.subplots(figsize=(10, 6))
    xgb.plot_importance(model, importance_type=importance_type, ax=ax)
    plt.title(f"Feature Importance ({importance_type})")
    plt.tight_layout()
    plt.close(fig)
    return fig

def plot_residuals(model, dvalid, valid_y):
    """Plot residuals vs true values."""
    preds = model.predict(dvalid)
    residuals = valid_y - preds
    
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(valid_y, residuals, alpha=0.5)
    plt.axhline(y=0, color="r", linestyle="-")
    plt.title("Residuals vs True Values")
    plt.xlabel("True Values")
    plt.ylabel("Residuals")
    plt.tight_layout()
    plt.close(fig)
    return fig

## Optuna Objective Function with MLflow Child Runs

In [ ]:
def create_objective(dtrain, dvalid, valid_y):
    """Create Optuna objective function for a specific dataset."""
    def objective(trial):
        with mlflow.start_run(nested=True):
            params = {
                "objective": "reg:squarederror",
                "eval_metric": "rmse",
                "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
                "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
                "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            }
            
            if params["booster"] in ["gbtree", "dart"]:
                params["max_depth"] = trial.suggest_int("max_depth", 1, 9)
                params["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
                params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
                params["grow_policy"] = trial.suggest_categorical(
                    "grow_policy", ["depthwise", "lossguide"]
                )
            
            model = xgb.train(params, dtrain, num_boost_round=100)
            preds = model.predict(dvalid)
            mse = mean_squared_error(valid_y, preds)
            rmse = math.sqrt(mse)
            
            mlflow.log_params(params)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("rmse", rmse)
            
        return mse
    return objective

In [ ]:
def champion_callback(study, trial):
    """Log when a new best trial is found."""
    if study.best_trial.number == trial.number:
        print(f"  Trial {trial.number}: {trial.value:.4f} (new best)")

optuna.logging.set_verbosity(optuna.logging.WARNING)

## Run Hyperparameter Optimization Per Category

In [ ]:
# Store results for each category
category_results = {}

for category, splits in category_splits.items():
    print(f"\n{'='*50}")
    print(f"Training model for: {category.upper()}")
    print(f"{'='*50}")
    
    run_name = f"{category}-optimization"
    
    with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
        # Create objective for this category
        objective = create_objective(
            splits["dtrain"],
            splits["dvalid"],
            splits["valid_y"],
        )
        
        # Run optimization
        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=n_trials, callbacks=[champion_callback])
        
        # Log best results
        mlflow.log_params(study.best_params)
        mlflow.log_metric("best_mse", study.best_value)
        mlflow.log_metric("best_rmse", math.sqrt(study.best_value))
        
        mlflow.set_tags({
            "project": "Electronics Sales Prediction",
            "category": category,
            "optimizer_engine": "optuna",
            "model_family": "xgboost",
        })
        
        # Train final model with best params
        best_params = study.best_params.copy()
        best_params["objective"] = "reg:squarederror"
        best_params["eval_metric"] = "rmse"
        best_model = xgb.train(best_params, splits["dtrain"], num_boost_round=100)
        
        # Log plots
        df_for_plot = category_datasets[category].copy()
        corr_plot = plot_correlation_with_target(df_for_plot)
        mlflow.log_figure(figure=corr_plot, artifact_file="correlation_plot.png")
        
        importance_plot = plot_feature_importance(best_model, best_params.get("booster", "gbtree"))
        mlflow.log_figure(figure=importance_plot, artifact_file="feature_importances.png")
        
        residual_plot = plot_residuals(best_model, splits["dvalid"], splits["valid_y"])
        mlflow.log_figure(figure=residual_plot, artifact_file="residuals.png")
        
        # Log model
        mlflow.xgboost.log_model(
            xgb_model=best_model,
            artifact_path="model",
            input_example=splits["train_x"].iloc[[0]],
        )
        
        model_uri = mlflow.get_artifact_uri("model")
        
        # Store results
        category_results[category] = {
            "study": study,
            "best_model": best_model,
            "model_uri": model_uri,
            "best_mse": study.best_value,
            "best_rmse": math.sqrt(study.best_value),
        }
        
        print(f"\n{category} - Best RMSE: {math.sqrt(study.best_value):.4f}")
        print(f"Model logged to: {model_uri}")

## Results Summary

In [ ]:
print("\n" + "="*50)
print("SUMMARY: Best Results Per Category")
print("="*50)

for category, results in category_results.items():
    study = results["study"]
    print(f"\n{category.upper()}")
    print(f"  Best trial: {study.best_trial.number}")
    print(f"  Best RMSE: {results['best_rmse']:.4f}")
    print(f"  Best params:")
    for k, v in study.best_params.items():
        print(f"    {k}: {v}")

## Optuna Visualizations (Select a Category)

In [ ]:
import optuna.visualization as vis

# Select category to visualize
viz_category = list(category_results.keys())[0]  # Change as needed
study = category_results[viz_category]["study"]
print(f"Visualizing: {viz_category}")

fig = vis.plot_optimization_history(study)
fig.show()

In [ ]:
fig = vis.plot_param_importances(study)
fig.show()

In [ ]:
fig = vis.plot_parallel_coordinate(study)
fig.show()

In [ ]:
fig = vis.plot_slice(study)
fig.show()

## Load and Use Models

In [ ]:
# Example: Load and use a specific category model
test_category = list(category_results.keys())[0]
model_uri = category_results[test_category]["model_uri"]
splits = category_splits[test_category]

loaded_model = mlflow.xgboost.load_model(model_uri)
predictions = loaded_model.predict(splits["dvalid"])

result_df = splits["valid_x"].copy()
result_df["actual"] = splits["valid_y"].values
result_df["predicted"] = predictions

print(f"Category: {test_category}")
print(f"Prediction stats:")
print(f"  Mean: {predictions.mean():.2f}")
print(f"  Std:  {predictions.std():.2f}")
result_df[["actual", "predicted"]].head(10)